# Préparation des données

In [44]:
# Chargment des bibliothèques
import pandas as pd
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import joblib


# Chargement du dataset
df = pd.read_csv("../data/raw/winequality-red.csv", sep=';')

## Gestion des valeurs manquantes
- Suppression de lignes ou colonnes trop incomplètes.
- Imputation (remplacement par la moyenne, la médiane, le mode…).
- Utilisation de modèles d’imputation plus avancés (KNNImputer, IterativeImputer).

> _Objectif_ : ne pas perturber l’apprentissage 


In [45]:
# Gestion des valeurs manquantes
df.isnull().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

- _Remarque_ : Ici, il n'y à aucune valeu manquante alors cette étape n'est pas nécéssaire.

## Séparation des données

- Données d'entrainement
- Données de test

In [46]:
# Séparation des features et de la cible
X = df.drop("quality", axis=1)
y = df["quality"]

# Séparation en train / test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2,          # 20% pour le test
    random_state=42,        # pour la reproductibilité
    stratify=y              # utile si classes déséquilibrées
)

print("Taille du jeu d'entraînement :", X_train.shape)
print("Taille du jeu de test :", X_test.shape)


Taille du jeu d'entraînement : (1279, 11)
Taille du jeu de test : (320, 11)


## Mise à l'échelle

- StandardScaler : moyenne = 0, écart-type = 1.
- MinMaxScaler : valeurs entre 0 et 1.

> _Objectif_ : adapter les données aux modèles sensibles aux différences d'échelle (ex. KNN, SVM, régression linéaire).

In [47]:
# Initialisation du scaler
scaler = StandardScaler()

# Fit uniquement sur le train, transform sur les deux
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Sélection des variables

- Corrélations fortes
- Importances des features avec des modèles simples
- Techniques avancées telles que la PCA

> _Objectif_ : Identifier les variables les plus pertinentes, réduire les features pour accélérer l'entrainement du modèle.

- _Remarque_ : Le jeu de données étant de taille modérée (11 features), il n'est pas nécéssaire de réaliser une data selection. 

Celà dit, voici un tableau récapitulatif des méthodes de séléction : 

| Méthode                             | Principe                                            | Exemple pratique                                                     |
| ----------------------------------- | --------------------------------------------------- | -------------------------------------------------------------------- |
| **Filtrage statistique**            | Retire les variables peu corrélées à la cible       | `SelectKBest`, test de corrélation                                   |
| **Wrapper (sélectif)**              | Teste différentes combinaisons via cross-validation | `RFE` (Recursive Feature Elimination)                                |
| **Basé sur l’importance du modèle** | Utilise les scores de feature importance            | `RandomForest.feature_importances_`, `PermutationImportance`, `SHAP` |


## Sauvergarde du scaler et du split

- Pour de futures utilisations
- Faciliter le documentation 

In [48]:
# Sauvegarde des splits pour fixer la seed du randomstate
joblib.dump((X_train, X_test, y_train, y_test), "../data/processed/splits.pkl")

['../data/processed/splits.pkl']

In [49]:
# Sauvegarde du scaler
joblib.dump(scaler, "../models/scaler.joblib")

['../models/scaler.joblib']